In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from holidays.countries import US
import numpy as np

### DEFINE
    設定起始年/月 & 檔案位置

In [2]:
# 讀取檔案位置

# 初始年月份
set_data_year = 2022
set_data_month = 1
end_year = 2022
end_month = 12

# 檔案位置
Site = './data/'

# 檔案輸出位置
custom_output_path_dfTime = './dfTime/year'
custom_output_path_dfTLC = './dfTLC/year'

# 年月
start_year = set_data_year
start_month = set_data_month

datasite_fhvhv = []
datasite_yellow = []
for year in range(start_year, end_year+1):  # 修改为你的年份范围
    if year == start_year:
        if year < end_year :
            for month in range(start_month, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)                
        elif year == end_year:
            for month in range(start_month, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)    
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
    elif year > start_year:
        if year < end_year :
            for month in range(1, 13):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   
        elif year == end_year:
            for month in range(1, end_month+1):  # 生成1到12的月份
                filename = f'{Site}fhvhv_tripdata_{year}-{month:02}.parquet'
                datasite_fhvhv.append(filename)         
                filename = f'{Site}yellow_tripdata_{year}-{month:02}.parquet'
                datasite_yellow.append(filename)   


print("done時間:", datetime.datetime.now())

done時間: 2023-08-15 19:48:01.854787


### 節日

In [3]:
from holidays.calendars import gregorian
from holidays.calendars.gregorian import _get_nth_weekday_of_month
class AllHolidays(US):
    def _populate(self, year):
        super()._populate(year)
        self.pop_named("Lincoln's Birthday")
        self.pop_named("Susan B. Anthony Day")
        
        self._add_holiday("Valentine's Day", 2, 14)
        self._add_holiday("St.Patrick's Day", 3, 17)
        self._add_holiday("Halloween", 10, 31)
     
        self._add_holiday("Christmas Eve", 12, 24)
        self._add_holiday("Christmas season", 12, 26)
        self._add_holiday("Christmas season", 12, 27)
        self._add_holiday("Christmas season", 12, 28)
        self._add_holiday("Christmas season", 12, 29)
        self._add_holiday("Christmas season", 12, 30)
        
        self._add_holiday("New Year's Eve", 12, 31)

# 實例化AllHolidays
ny_holidays_all = AllHolidays(subdiv='NY', years=[2019, 2020, 2021, 2022, 2023])

# 新增2019~2023年的復活節、復活節星期一
ny_holidays_all[datetime.date(2019, 4, 21)] = "Easter"
ny_holidays_all[datetime.date(2019, 4, 22)] = "Easter Monday"
ny_holidays_all[datetime.date(2020, 4, 12)] = "Easter"
ny_holidays_all[datetime.date(2020, 4, 13)] = "Easter Monday"
ny_holidays_all[datetime.date(2021, 4, 4)] = "Easter"
ny_holidays_all[datetime.date(2021, 4, 5)] = "Easter Monday"
ny_holidays_all[datetime.date(2022, 4, 17)] = "Easter"
ny_holidays_all[datetime.date(2022, 4, 18)] = "Easter Monday"
ny_holidays_all[datetime.date(2023, 4, 9)] = "Easter"
ny_holidays_all[datetime.date(2023, 4, 10)] = "Easter Monday"

# 新增2019~2023感恩節
ny_holidays_all[datetime.date(2019,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 29)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2019,11, 30)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2020,11, 28)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 26)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2021,11, 27)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 25)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2022,11, 26)] =  "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 23)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 24)] = "Thanksgiving weekend"
ny_holidays_all[datetime.date(2023,11, 25)] = "Thanksgiving weekend"


### 執行
    一年資料約10分鐘

In [4]:
# 記錄開始時間
start_time = time.time()

###  留下需要特徵 & 移除不在當月資料
# 建立一個空的DataFrame來存放合併後的數據
df_fhvhv = pd.DataFrame()
df_yellow = pd.DataFrame()
#年月設定
data_year = set_data_year
data_month = set_data_month
# 逐一讀取並合併所有Parquet檔案
for file in datasite_fhvhv:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    # 留下需要的欄位
    df =  df[['hvfhs_license_num','pickup_datetime','dropoff_datetime','PULocationID','DOLocationID','trip_miles','trip_time','shared_match_flag']]
    # 移除非當月的data
    df =  df[(df['pickup_datetime'].dt.year == data_year) & (df['pickup_datetime'].dt.month == data_month)]
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_fhvhv = pd.concat([df_fhvhv, df], ignore_index=True)    
data_year = set_data_year
data_month = set_data_month
for file in datasite_yellow:
    # 使用pandas的read_parquet函數讀取Parquet檔案並將其轉換為DataFrame
    df = pd.read_parquet(file)
    # 留下需要的欄位
    df =  df[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
    # 移除非當月的data
    df =  df[(df['tpep_pickup_datetime'].dt.year == data_year) & (df['tpep_pickup_datetime'].dt.month == data_month)]
    # 年月處理
    if data_month == 12:
        data_year += 1
        data_month = 1
    else :
        data_month += 1
    # 將數據合併到已有的DataFrame中
    df_yellow = pd.concat([df_yellow, df], ignore_index=True)    
#清理暫存
del data_month,data_year,df    
    
###  特徵整理,統一命名
df_TLC = pd.DataFrame()
#名稱統一
rename_dict_fhvhv = {'hvfhs_license_num':'Name','pickup_datetime': 'BoardingTime', 'dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_miles': 'TripMiles', 'trip_time': 'TripTime', 'base_passenger_fare': 'BaseFare','shared_match_flag':'shared_match_flag'}
df_fhvhv.rename(columns=rename_dict_fhvhv, inplace=True)
rename_dict_yellow = {'tpep_pickup_datetime': 'BoardingTime', 'tpep_dropoff_datetime': 'DisembarkingTime', 'PULocationID': 'PULocationID','DOLocationID':'DOLocationID', 'trip_distance': 'TripMiles', 'fare_amount': 'BaseFare'}
df_yellow.rename(columns=rename_dict_yellow, inplace=True)
#fhvhv_Name
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0003', 'Name'] = 'uber'
df_fhvhv.loc[df_fhvhv['Name'] == 'HV0005', 'Name'] = 'lyft'
#yellow 新增特徵 TripTime,Name
df_yellow['TripTime'] = (df_yellow['DisembarkingTime'] - df_yellow['BoardingTime']).dt.total_seconds()
df_yellow['Name'] = 'yellow'
df_yellow['shared_match_flag'] = 'N'
#yellow合併到fhvhv
df_yellow = df_yellow.reindex(columns = df_fhvhv.columns)
df_TLC = pd.concat([df_fhvhv, df_yellow], ignore_index=True)  
#清理暫存
del df_yellow ,df_fhvhv


###移除非目標車商
df_TLC = df_TLC.drop(df_TLC[df_TLC['Name'] == 'HV0004'].index)

    
###  移除上車地點 PULocationID 未知 (264,265)
df_TLC = df_TLC[~df_TLC['PULocationID'].isin([264, 265])]


###  移除不符合的共乘資料
# 定義起始日期和結束日期
uber_start_date = pd.to_datetime('2020-03-17')
uber_end_date = pd.to_datetime('2022-06-20')
lyft_start_date = pd.to_datetime('2020-03-17')
lyft_end_date = pd.to_datetime('2021-08-18')
lyft_start_date1 = pd.to_datetime('2023-05-12')
df_TLC = df_TLC[~ ((df_TLC['Name'] == 'uber') & (df_TLC['shared_match_flag'] == 'Y') & (df_TLC['BoardingTime'].between(uber_start_date, uber_end_date)))]
df_TLC = df_TLC[~ ((df_TLC['Name'] == 'lyft') & (df_TLC['shared_match_flag'] == 'Y') & ((df_TLC['BoardingTime'].between(lyft_start_date, lyft_end_date))|(df_TLC['BoardingTime'] >= lyft_start_date1 )))]
#清理暫存
del uber_start_date ,uber_end_date,lyft_start_date,lyft_end_date,lyft_start_date1


###  移除 TripTime <= 0
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripTime'] <= 0].index)


###  移除TripMiles <= 0
df_TLC = df_TLC.drop(df_TLC[df_TLC['TripMiles'] <= 0].index)


### 增加所有節日標籤
df_TLC['is_holiday'] = np.where(df_TLC['BoardingTime'].dt.date.isin(ny_holidays_all), True, False)
df_TLC['is_holiday'].value_counts()


###  新增星期特徵&年月日時切分
df_TLC['weekday'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.weekday
df_TLC['date'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.date
df_TLC['year'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.year
df_TLC['month'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.month
df_TLC['day'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.day
df_TLC['hour'] = (pd.to_datetime(df_TLC['BoardingTime'])).dt.hour

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 1022.65 秒
done時間: 2023-08-15 20:05:08.219359


### 時間序列
    這邊需要確認時間序列的 COLUMN (可更改)

In [5]:
# 記錄開始時間
start_time = time.time()

dfTime_TLC = df_TLC.groupby(['Name','year','month','day',
                     'hour','weekday','PULocationID','is_holiday'
                  ]).size().reset_index(name="次數")

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 71.27 秒
done時間: 2023-08-15 20:06:31.270162


### 輸出為一份新檔案
    這邊需要確認檔案輸出位置 custom_output_path (可在DEFINE更改)

In [6]:
#時間序列
# 記錄開始時間
start_time = time.time()

#檔案輸出位置
custom_output_path = custom_output_path_dfTime

dfTime_TLC.to_parquet(f'{custom_output_path}/dfTime_TLC-{set_data_year}.parquet')

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 2.1 秒
done時間: 2023-08-15 20:06:36.915148


In [7]:
#整年檔案
# 記錄開始時間
start_time = time.time()

#檔案輸出位置
custom_output_path = custom_output_path_dfTLC

df_TLC.to_parquet(f'{custom_output_path}/TLC-{set_data_year}.parquet')

# 記錄結束時間
end_time = time.time()
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

程式執行花費的時間： 182.51 秒
done時間: 2023-08-15 20:09:43.558180


### 查看dateframe

In [8]:
df_TLC

,Name,BoardingTime,DisembarkingTime,PULocationID,DOLocationID,TripMiles,TripTime,shared_match_flag,is_holiday,weekday,date,year,month,day,hour
0,uber,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,5.26,923.0,N,True,4,2021-01-01,2021,1,1,0
1,uber,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,3.65,1382.0,N,True,4,2021-01-01,2021,1,1,0
2,uber,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,3.51,849.0,N,True,4,2021-01-01,2021,1,1,0
3,uber,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,0.74,179.0,N,True,4,2021-01-01,2021,1,1,0
4,uber,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,9.20,1228.0,N,True,4,2021-01-01,2021,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205499501,yellow,2021-12-31 23:18:02,2021-12-31 23:32:19,145,107,3.99,857.0,N,True,4,2021-12-31,2021,12,31,23
205499502,yellow,2021-12-31 23:46:00,2022-01-01 00:07:00,236,158,5.96,1260.0,N,True,4,2021-12-31,2021,12,31,23
205499503,yellow,2021-12-31 23:46:00,2022-01-01 00:04:00,48,13,4.99,1080.0,N,True,4,2021-12-31,2021,12,31,23
205499504,yellow,2021-12-31 23:53:00,2022-01-01 00:02:00,75,75,1.36,540.0,N,True,4,2021-12-31,2021,12,31,23


In [8]:
dfTime_TLC

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2021,1,1,0,4,3,True,15
1,lyft,2021,1,1,0,4,4,True,41
2,lyft,2021,1,1,0,4,6,True,4
3,lyft,2021,1,1,0,4,7,True,64
4,lyft,2021,1,1,0,4,9,True,2
...,...,...,...,...,...,...,...,...,...
827143,yellow,2021,2,28,23,6,257,False,1
827144,yellow,2021,2,28,23,6,260,False,2
827145,yellow,2021,2,28,23,6,261,False,2
827146,yellow,2021,2,28,23,6,262,False,2
